In [ ]:
!pip install transformers datasets wandb
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import os
cache_dir_path = "/content/hf-cache"
os.environ["HF_HOME"] = "/content/hf-cache"
os.environ["WANDB_PROJECT"] = 'distilbert-arpit-ft-code-1024-v2'

In [ ]:
import torch
if not torch.cuda.is_available():
    print("Warning: GPU is not enabled. Go to Runtime > Change runtime type > GPU.")
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arpitkaple13 (arpitkaple13-rbu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from datasets import Dataset, load_from_disk
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
clf_ds = load_from_disk('/content/drive/MyDrive/major_project_models/dataset_tokenized_distilbert/train')
val_clf_ds = load_from_disk('/content/drive/MyDrive/major_project_models/dataset_tokenized_distilbert/valid')
test_clf_ds = load_from_disk('/content/drive/MyDrive/major_project_models/dataset_tokenized_distilbert/test')

def preprocess_function(examples):
    return tokenizer(examples["code"], truncation=True, padding=True)
tokenized_clf = clf_ds.map(preprocess_function, batched=True)
tokenized_val_clf = val_clf_ds.map(preprocess_function, batched=True)
tokenized_test_clf = test_clf_ds.map(preprocess_function, batched=True)

def rename_cols(example):
    return {"text": example['code'], "label": example['y']}
tokenized_clf_rename = tokenized_clf.map(rename_cols, remove_columns=['id', 'y', 'idx', 'code', 'seq_len'])
val_tokenized_clf_rename = tokenized_val_clf.map(rename_cols, remove_columns=['id', 'y', 'idx', 'code', 'seq_len'])
test_tokenized_clf_rename = tokenized_test_clf.map(rename_cols, remove_columns=['id', 'y', 'idx', 'code', 'seq_len'])

Map:   0%|          | 0/36750 [00:00<?, ? examples/s]

Map:   0%|          | 0/36750 [00:00<?, ? examples/s]

In [ ]:
test_tokenized_clf_rename

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'label'],
    num_rows: 6023
})

In [ ]:
tokenized_clf_rename

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'label'],
    num_rows: 36750
})

In [ ]:
val_tokenized_clf_rename

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'label'],
    num_rows: 5956
})

In [ ]:
from datasets import DatasetDict

dataset = DatasetDict()
dataset['train'] = tokenized_clf_rename
dataset['val'] = val_tokenized_clf_rename
dataset['test'] = test_tokenized_clf_rename

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 36750
    })
    val: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 5956
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6023
    })
})

In [ ]:
from transformers import AutoModel
import torch

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [ ]:
dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
dataset_hidden = dataset.map(extract_hidden_states, batched=True, batch_size=8)
dataset_hidden["test"].column_names

Map:   0%|          | 0/36750 [00:00<?, ? examples/s]

Map:   0%|          | 0/5956 [00:00<?, ? examples/s]

Map:   0%|          | 0/6023 [00:00<?, ? examples/s]

['text', 'input_ids', 'attention_mask', 'label', 'hidden_state']

In [ ]:
from transformers import AutoModelForSequenceClassification
num_labels = 2
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 8
gradient_accumulation_steps = 8
# logging_steps = len(emotions_encoded["train"]) // batch_size
# Previous model: f"{model_ckpt}-finetuned-emotion"
model_name = f"/content/drive/MyDrive/major_project_models/{model_ckpt}-distilbert-1024"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  gradient_accumulation_steps=gradient_accumulation_steps,
                                  fp16=True,
                                  weight_decay=0.01,
                                  evaluation_strategy="steps",
                                  eval_steps=50,
                                  disable_tqdm=False,
                                  logging_steps=50,
                                  report_to='wandb',
                                  push_to_hub=False,
                                  log_level="error",
                                  max_steps = 1280
                                  )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  train_dataset=dataset_hidden["train"],
                  eval_dataset=dataset_hidden["val"],
                  tokenizer=tokenizer)

<ipython-input-28-c5b5fe50f56d>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args,


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.366000,0.347864,0.842512,0.813675
100,0.355800,0.343723,0.853761,0.790489
150,0.348400,0.333503,0.815648,0.815823
200,0.341400,0.355571,0.835964,0.814693
250,0.338600,0.346274,0.837643,0.814554
300,0.336700,0.337045,0.844023,0.812657
350,0.356900,0.335649,0.850403,0.803475
400,0.329600,0.333600,0.825386,0.821678
450,0.348800,0.340018,0.835796,0.814575
500,0.328200,0.344246,0.810779,0.815785


TrainOutput(global_step=1280, training_loss=0.33455891609191896, metrics={'train_runtime': 1603.9709, 'train_samples_per_second': 51.073, 'train_steps_per_second': 0.798, 'total_flos': 1.083848255803392e+16, 'train_loss': 0.33455891609191896, 'epoch': 2.2263822377013494})